In [7]:
!pip install rouge
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [8]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re

from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm

In [9]:
# # Initializing a BART facebook/bart-large style configuration
# configuration = BartConfig()

# # Initializing a model (with random weights) from the facebook/bart-large style configuration
# model = BartModel(configuration)

# # Accessing the model configuration
# configuration = model.config

In [10]:
# model_checkpoints = "gogamza/kobart-base-v2"
# model_config = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 
# model_config.config

In [11]:
from transformers import (
    BartForConditionalGeneration, BartTokenizer, BartForCausalLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split


# ## Initiating model and trainer for training
from transformers import BartModel, BartConfig
from transformers import BartTokenizerFast



In [12]:
config = BartConfig.from_pretrained("gogamza/kobart-base-v2")
config

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "kobart_version": 2.0,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIV

In [13]:
model = BartForConditionalGeneration(config)
tokenizer = BartTokenizerFast.from_pretrained("gogamza/kobart-base-v2")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [14]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[ 0.0040, -0.0101, -0.0007,  ...,  0.0051,  0.0125,  0.0212],
        [-0.0194, -0.0012, -0.0108,  ...,  0.0102, -0.0066, -0.0036],
        [ 0.0220, -0.0087,  0.0095,  ...,  0.0383, -0.0026, -0.0093],
        ...,
        [-0.0074,  0.0031,  0.0044,  ..., -0.0038,  0.0075,  0.0232],
        [-0.0124, -0.0278,  0.0031,  ..., -0.0206, -0.0030, -0.0006],
        [-0.0017, -0.0517, -0.0072,  ...,  0.0055,  0.0316, -0.0058]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0040, -0.0136,  0.0233,  ...,  0.0471, -0.0266,  0.0432],
        [-0.0129,  0.0167,  0.0271,  ..., -0.0117,  0.0195, -0.0037],
        [-0.0185,  0.0194,  0.0639,  ...,  0.0023,  0.0187,  0.0005],
        ...,
        [-0.0614,  0.0025,  0.0357,  ...,  0.0220,  0.0063,  0.0040],
        [ 0.0228, -0.0063,  0.0269,  ...,  0.0089, -0.0324,  0.0013],
        [-0.0323,  0.0393,  0.0041,  ...,  0.0059,  0.0115, -0.0261]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.

In [43]:
#from transformers import BartConfig, BartModel

#configuration = BartConfig() # default Bart config
#model = BartModel(configuration) # default randomly initialised BART

In [44]:
#configuration

In [45]:
#model_checkpoints = "gogamza/kobart-base-v2"
#tokenizer = BartTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
#model_config = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2") 
# print(model_config.config)
# configuration = BartConfig()
#configuration = BartConfig()
#model = BartModel(model_config.config)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [15]:
model_config.config

NameError: name 'model_config' is not defined

In [ ]:
#model=BartForCausalLM(model_config.config)

In [ ]:
#BartConfig(model_config.config)

In [ ]:
#model_config

In [ ]:

#model = BartForCausalLM(model_config.config)

In [ ]:
for i in model.parameters():
    print(i)

In [ ]:
# for i in model.parameters():
#     print(i)

In [ ]:

# configuration = BartConfig(
#     vocab_size=30000,
#     max_position_embeddings=1026,
#     d_model=768,
#     encoder_layers=6,
#     decoder_layers=6,
#     encoder_attention_heads=16,
#     decoder_attention_heads=16,
#     decoder_ffn_dim=3072,
#     encoder_ffn_dim=3072,
#     #bos_token_id=1,
#     #decoder_start_token_id=1,
#     #eos_token_id=1
# )
# model = BartForCausalLM(configuration)
# model_checkpoints = "gogamza/kobart-base-v2"
# tokenizer = BartTokenizerFast.from_pretrained(model_checkpoints)

# #tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)


In [ ]:
tokenizer

In [ ]:
model

# fine-tuning

# pretrained

In [ ]:
model.config

In [ ]:
# 이것도 됨
# model_checkpoints = "alaggung/bart-pretrained"

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (..) 제거
    #sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    # 스폐셜 토큰 적용할 거면 여기 위에 영어 외 문자 공백으로 만들때 스폐셜 토큰을 넘어갈 수 있도록 지정해주면된다.
    # 그리고 세번째 정규표현식을 지워야 ||할 것이다. 
    return sentence

In [ ]:
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [ ]:
train_Category = train_df['Category'].unique()
val_Category = val_df['Category'].unique()
def categori_ext(data, Category, tv):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
    return df

In [ ]:
train_df = categori_ext(train_df, train_Category, 'train')
val_df = categori_ext(val_df, val_Category, 'val')

In [ ]:
train_df.head()

In [ ]:
# 전체 Text 데이터에 대한 전처리 (1)
train_text = []
val_text = []

for tt in tqdm(train_df['Text']):
    train_text.append(preprocess_sentence(tt))

for vt in tqdm(val_df['Text']):
    val_text.append(preprocess_sentence(vt))

In [ ]:
train_df = pd.DataFrame(zip(train_text), columns=['Text'])
val_df = pd.DataFrame(zip(val_text), columns=['Text'])

In [ ]:
train_df.head()

In [ ]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)

In [ ]:
print(train_data)
print(val_data)

In [ ]:
max_input = 128
max_target = 128
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [ ]:
 def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [16]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [17]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Text'])):
        label_id.append(tokenizer.encode(data_to_process['Text'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Text'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Text'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [18]:
def data_collator(df):
    text = []
    for vt in tqdm(df):
        text.append(vt)

    batch = tokenizer.prepare_seq2seq_batch(src_texts=text, max_length=256, padding='max_length')
    batch["labels"] = batch["input_ids"].copy()
    for k in batch:
        batch[k] = torch.tensor(batch[k])
    return batch

In [19]:
"""
special_words = [
                "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
                "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
                "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
                "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
                ]

tokenizer.add_special_tokens({"additional_special_tokens": special_words})
model.resize_token_embeddings(len(tokenizer))
"""

model.config.max_length = 128 
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5



In [20]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text'])

NameError: name 'train_data' is not defined

In [21]:
val_tokenize_data

NameError: name 'val_tokenize_data' is not defined

In [22]:
model.config

BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "kobart_version": 2.0,
  "label2id": {
    "

In [23]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)
   

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/non_MLM_test",
    num_train_epochs=10, # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch'


    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [25]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [26]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
   # compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

NameError: name 'train_tokenize_data' is not defined

In [27]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
trainer.evaluate()

In [ ]:
#encoding=utf-8

from transformers import (
    BartForConditionalGeneration, BartTokenizer, BartForCausalLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split


# ## Initiating model and trainer for training
from transformers import BartModel, BartConfig
from transformers import BartTokenizerFast


configuration = BartConfig(
    vocab_size=52000,
    max_position_embeddings=258,
    d_model=256,
    encoder_layers=3,
    decoder_layers=3,
    encoder_attention_heads=4,
    decoder_attention_heads=4,
    decoder_ffn_dim=1024,
    encoder_ffn_dim=1024,
)
model = BartForCausalLM(configuration)
model_checkpoints = "gogamza/kobart-base-v2"
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoints)

# tokenizer = BartTokenizerFast.from_pretrained(model_checkpoints)

# ### HTTP Request DataPreparing & Modeling
data = []
# with open("data/nosp_train_text.csv") as f1:
#     for src in f1:
#       data.append(
#           {
#               "seq2seq": {
#                   "input": src.strip()
#               }
#           }
#       )
#print(f'total size of data is {len(data)}')
train_textfile_path = "data/nosp_train_text.csv"


with open(train_textfile_path, encoding="utf-8") as f:
    data.append(train_textlines) = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]


# splitting dataset into train, validation
split = 0.2
train_dataset, eval_dataset = random_split(data, lengths=[int((1-split)*len(data))+1, int(split*len(data))])


# defining collator functioon for preparing batches on the fly ..
def data_collator(features:list):
   inputs = [f["seq2seq"]["input"] for f in features]
   batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, max_length=256, padding='max_length')
   batch["labels"] = batch["input_ids"].copy()
   for k in batch:
        batch[k] = torch.tensor(batch[k])
   return batch


batch_out = data_collator(eval_dataset)
print(batch_out)
print(batch_out['input_ids'].shape,batch_out['labels'].shape,batch_out['attention_mask'].shape)


# defining training related arguments
args = Seq2SeqTrainingArguments(output_dir="clm-checkpoints",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=8,
                        per_device_eval_batch_size=8,
                        learning_rate=5e-5,
                        num_train_epochs=1,
                        logging_dir="./logs")


# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset, 
                eval_dataset=eval_dataset)


# ## Training time
trainer.train()
# It will take hours to train this model on this dataset


# lets save model
trainer.evaluate(eval_dataset=eval_dataset)
trainer.save_model("clm-checkpoints")

In [ ]:
model.config

In [ ]:
for i in model.parameters():
    print(i)

In [ ]:
from transformers import BartTokenizer, BartForCausalLM

tokenizer = BartTokenizer.from_pretrained(model_checkpoints)
model = BartForCausalLM.from_pretrained(model_checkpoints, add_cross_attention=False)

In [ ]:
for i in model.parameters():
    print(i)

In [ ]:
#encoding=utf-8

from transformers import (
    BartForConditionalGeneration, BartTokenizer, BartForCausalLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split


# ## Initiating model and trainer for training
from transformers import BartModel, BartConfig
from transformers import BartTokenizerFast

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 
configuration = BartConfig(model)
    
#     vocab_size=52000,
#     max_position_embeddings=258,
#     d_model=256,
#     encoder_layers=3,
#     decoder_layers=3,
#     encoder_attention_heads=4,
#     decoder_attention_heads=4,
#     decoder_ffn_dim=1024,
#     encoder_ffn_dim=1024,
#)
model = BartForCausalLM(configuration)





In [ ]:
model.config

In [36]:
#encoding=utf-8

from transformers import (
    BartForConditionalGeneration, BartTokenizer, BartForCausalLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split


# ## Initiating model and trainer for training
from transformers import BartModel, BartConfig
from transformers import BartTokenizerFast


configuration = BartConfig(
    vocab_size=30000,
    max_position_embeddings=1026,
    d_model=768,
    encoder_layers=6,
    decoder_layers=6,
    encoder_attention_heads=16,
    decoder_attention_heads=16,
    decoder_ffn_dim=3072,
    encoder_ffn_dim=3072,
    bos_token_id=1,
    decoder_start_token_id=1,
    eos_token_id=1
)
model = BartForCausalLM(configuration)
model_checkpoints = "gogamza/kobart-base-v2"
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoints)

# tokenizer = BartTokenizerFast.from_pretrained(model_checkpoints)

# ### HTTP Request DataPreparing & Modeling


loading file vocab.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/vocab.json
loading file merges.txt from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/merges.txt
loading file tokenizer.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/tokenizer.json
loading file added_tokens.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/added_tokens.json
loading file special_tokens_map.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/special_tokens_map.json
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub

In [37]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (..) 제거
    #sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    # 스폐셜 토큰 적용할 거면 여기 위에 영어 외 문자 공백으로 만들때 스폐셜 토큰을 넘어갈 수 있도록 지정해주면된다.
    # 그리고 세번째 정규표현식을 지워야 할 것이다. 
    return sentence

In [38]:
import pandas as pd
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [39]:
train_Category = train_df['Category'].unique()
val_Category = val_df['Category'].unique()
def categori_ext(data, Category, tv):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
    return df

In [40]:
train_df = categori_ext(train_df, train_Category, 'train')
val_df = categori_ext(val_df, val_Category, 'val')

In [41]:
train_df.head()

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)


In [42]:
# 전체 Text 데이터에 대한 전처리 (1)
train_text = []
val_text = []
from tqdm import tqdm
import re
for tt in tqdm(train_df['Text']):
    train_text.append(preprocess_sentence(tt))

for vt in tqdm(val_df['Text']):
    val_text.append(preprocess_sentence(vt))

100%|██████████| 1746/1746 [00:00<00:00, 34718.56it/s]


In [43]:
split = 0.2
train_dataset, eval_dataset = random_split(train_text, lengths=[int((1-split)*len(train_text))+1, int(split*len(train_text))])

In [44]:
len(train_dataset)

11196

In [54]:
def data_collator(df):
    text = []
    for vt in tqdm(df):
        text.append(vt)

    batch = tokenizer.prepare_seq2seq_batch(src_texts=text, max_length=256, padding='max_length')
    batch["labels"] = batch["input_ids"].copy()
    for k in batch:
        batch[k] = torch.tensor(batch[k])
    return batch

In [55]:
eval_dataset = data_collator(eval_dataset)
train_dataset = data_collator(train_dataset)

print(batch_out)
print(batch_out['input_ids'].shape,batch_out['labels'].shape,batch_out['attention_mask'].shape)

100%|██████████| 3/3 [00:00<00:00, 6367.87it/s]

{'input_ids': tensor([[11926, 15249, 13363,  ...,     3,     3,     3],
        [ 9545,  9698, 14334,  ...,     3,     3,     3],
        [ 9694,  9489, 11224,  ...,     3,     3,     3],
        ...,
        [11696, 15584, 10496,  ...,     3,     3,     3],
        [12191, 12719, 23901,  ...,     3,     3,     3],
        [17886, 12005, 29068,  ...,     3,     3,     3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[11926, 15249, 13363,  ...,     3,     3,     3],
        [ 9545,  9698, 14334,  ...,     3,     3,     3],
        [ 9694,  9489, 11224,  ...,     3,     3,     3],
        ...,
        [11696, 15584, 10496,  ...,     3,     3,     3],
        [12191, 12719, 23901,  ...,     3,     3,     3],
        [17886, 12005, 29068,  ...,     3,     3,     3]])}
torch.Size([1

In [56]:
args = Seq2SeqTrainingArguments(output_dir="clm-checkpoints",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="steps",
                        per_device_train_batch_size=4,
                        per_device_eval_batch_size=4,
                        learning_rate=3e-5,
                        num_train_epochs=1,
                        logging_dir="./logs")

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [57]:
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset, 
                eval_dataset=eval_dataset)

In [58]:
trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1
  Number of trainable parameters = 80541696
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
100%|██████████| 3/3 [00:00<00:00, 16384.00it/s]


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
data = []
# with open("../data/sample.txt") as f1:
#     for src in f1:
#       data.append(
#           {
#               "seq2seq": {
#                   "input": src.strip()
#               }
#           }
#       )
# print(f'total size of data is {len(data)}')
train_textfile_path = "data/nosp_train_text.csv"
train_summaryfile_path = "data/nosp_train_summary.csv"

with open(train_textfile_path, encoding="utf-8") as f:
            train_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(train_summaryfile_path, encoding="utf-8") as f:
            train_sumlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]  

# splitting dataset into train, validation



# defining collator functioon for preparing batches on the fly ..






# defining training related arguments
args = Seq2SeqTrainingArguments(output_dir="clm-checkpoints",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=8,
                        per_device_eval_batch_size=8,
                        learning_rate=5e-5,
                        num_train_epochs=1,
                        logging_dir="./logs")


# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset, 
                eval_dataset=eval_dataset)


# ## Training time
trainer.train()
# It will take hours to train this model on this dataset


# lets save model
trainer.evaluate(eval_dataset=eval_dataset)
trainer.save_model("clm-checkpoints")